# Generalized Assignment Problem
In this session, we will use the PuLP library to solve the Generalized Assignment Problem. GAP has the goal of minimizing costs or maximizing profits by assigning n tasks to m servers that have limited capacity and **each task can only be done by one server**. For more details, you can read the book **Sarker and Newton - Optimization Modeling, A Practical Approach**

**The following parameters are used:**

- n = number of tasks/jobs
- m = number of servers/workers
- Cij = cost for assigning job i to server j
- bj = available capacity/resources on server j
- aij = number of resources required to perform work i by server j

While the variable used is **Xij** which is a binary type variable (1 = job i done by server j, 0 = otherwise)

**Here is the objective function along with the constraints:**
<img src="https://raw.githubusercontent.com/rianromad/Image-Storage/main/gap%20obj%20ctr.PNG?token=AOWKXL7UQLTD5ZUMSMD36YTBBDKJI" />
*Source: Sarker and Newton - Optimization Modelling, A Practical Approach*


**Constraints that must be met are:**
1. Each task/job can only be done by one server
2. The number of resources used by vendors on some jobs must be less than or equal to their capacity

**Illustration of Problem:**

An agency will hold a celebration event by providing 9 types of food (menus). The agency has data on the names of vendors and the number of employees (vendors), food ordering costs (costs), and the number of employees at each vendor and menu (resources). How is the allocation of these vendors so that the costs incurred by the agency can be minimized?

The data used is sourced from the course I studied, Optimization Mathematics with Mr
Setyo Tri Windras Mara. In this course, I solved this problem using the Gurobi Python library. But this time I will solve it using the PuLP library.


### 1. Importing Library

In [1]:
from pulp import *
import pandas as pd
import numpy as np

### 2. Mengimpor Data
Data yang digunakan memiliki format .xlsx dengan 4 sheet. Saya menggunakan tambahan parameter **header** dan **usecols** untuk merapikan tabel tersebut. Saya juga menyamakan indeks untuk kemudahan akses pada tahap selanjutnya. Dapat dilihat bentuk masing-masing tabel yang akan digunakan. 

In [2]:
#load the data
menus = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Menus", header=0).set_index('Menu')
vendors = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Vendors", header=0).set_index('Vendor')
costs = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Costs", header=1,usecols=[2,3,4,5,6])
resources = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Resources", header=1,usecols=[2,3,4,5,6])

#create menu index in costs and resources table
costs['menu'] = menus.index
costs.set_index('menu', inplace=True)

resources['menu'] = menus.index
resources.set_index('menu', inplace=True)

#### a. Menus
The menus table only consists of the menu index along with the menu name.

In [3]:
menus

,Name
Menu,
1,Bakso
2,Soto
3,Nasi Sayur
4,Empal Gentong
5,Zuppa Soup
6,Kambing Guling
7,Selat Solo
8,Snack
9,Bebek Peking


#### b. Vendors
The vendors table contains the vendor index, vendor name and number of employees owned.


In [4]:
vendors

,Vendor Name,Number of Employees
Vendor,,
1,Sami Asih Group,20
2,Sumber Vitamin,30
3,Katineung,20
4,Murni,15
5,Sangkuriang,15


#### c. Costs
The costs table contains the costs required for each menu and its vendor.

In [5]:
costs

,1,2,3,4,5
menu,,,,,
1,197,114,173,175,137
2,184,108,148,188,147
3,151,156,122,124,118
4,200,109,111,132,157
5,110,130,142,129,182
6,171,159,200,171,125
7,114,200,112,185,138
8,150,165,156,136,100
9,184,129,193,199,148


#### d. Resources
This table contains the number of employees required at each vendor and menu.

In [6]:
resources

,1,2,3,4,5
menu,,,,,
1,5,5,2,5,2
2,2,4,5,1,5
3,5,1,4,4,1
4,5,3,4,1,4
5,5,5,2,3,4
6,3,4,4,3,2
7,4,5,2,2,3
8,2,5,4,3,2
9,3,4,4,5,3


### 3. Adding Variabel
In addition to creating the variables used, I also created some parameters for easy data access.

In [7]:
#create parameters
vendors_list = vendors.index
menus_list = menus.index
keys = [(i,j) for i in menus_list for j in vendors_list]

#variable
x = LpVariable.dicts("x", keys, 0, None, 'Binary')

### 4. Initialize the Model
This problem has the goal of minimizing costs so we use **LpMinimize**.

In [8]:
model = LpProblem('Choosing_Vendor', LpMinimize)

### 5. Adding Objective Function and Constraints


In [9]:
#Objective Function
model += lpSum([costs.loc[i,j]*x[i,j] for i in menus_list for j in vendors_list])

In [10]:
#constraints

#each job can only be done by one vendor
for i in menus_list:
    model += lpSum([x[i,j] for j in vendors_list]) == 1
    
#The number of employees used by vendors on some jobs must be less than or equal to their capacity
for j in vendors_list:
    model += lpSum([resources.loc[i,j]*x[i,j] for i in menus_list] ) <= vendors.loc[j,'Number of Employees']

### 6. Solve the Model and Check the Result

In [11]:
#if the status is equal to 1, it means that the model is optimal
model.solve()

1

In [12]:
#check the costs that must be issued by the agency
model.objective.value()

1025

In [13]:
#make a list containing solutions
solutions = []
for i,j in x.keys():
    if (x[i,j].varValue) == 1:
        solutions.append(x[i,j])

In [14]:
solutions

[x_(1,_2),
 x_(2,_2),
 x_(3,_5),
 x_(4,_2),
 x_(5,_1),
 x_(6,_5),
 x_(7,_3),
 x_(8,_5),
 x_(9,_2)]

From the list of solutions above, the first number shows the menu index, followed by the vendor index. It appears that the shape is still not good so it needs a little modification to access the numbers only.

In [15]:
vendor_sol = [int(str(solutions[j])[6]) for j in range(0,9)]
vendor_sol

[2, 2, 5, 2, 1, 5, 3, 5, 2]

I also made a dictionary on the menu and vendors to make the table easy to understand.

In [16]:
dict_menu = {i:j for (i,j) in zip(menus.index, list(menus['Name']))}
dict_menu

{1: 'Bakso',
 2: 'Soto',
 3: 'Nasi Sayur',
 4: 'Empal Gentong',
 5: 'Zuppa Soup',
 6: 'Kambing Guling',
 7: 'Selat Solo',
 8: 'Snack',
 9: 'Bebek Peking'}

In [17]:
dict_vendor = {i:j for (i,j) in zip(vendors.index, list(vendors['Vendor Name']))}
dict_vendor

{1: 'Sami Asih Group',
 2: 'Sumber Vitamin',
 3: 'Katineung',
 4: 'Murni',
 5: 'Sangkuriang'}

In [18]:
#create a table
df_sol = pd.DataFrame({'Menu': menus_list,
                       'Vendor':vendor_sol})

df_sol

,Menu,Vendor
0,1,2
1,2,2
2,3,5
3,4,2
4,5,1
5,6,5
6,7,3
7,8,5
8,9,2


The table above is still in the form of numbers only so it is not meaningful. Therefore  I convert these numbers using the dictionary that I created above.

In [19]:
df_sol['Menu'] = df_sol["Menu"].replace(dict_menu)
df_sol['Vendor'] = df_sol["Vendor"].replace(dict_vendor)

df_sol

,Menu,Vendor
0,Bakso,Sumber Vitamin
1,Soto,Sumber Vitamin
2,Nasi Sayur,Sangkuriang
3,Empal Gentong,Sumber Vitamin
4,Zuppa Soup,Sami Asih Group
5,Kambing Guling,Sangkuriang
6,Selat Solo,Katineung
7,Snack,Sangkuriang
8,Bebek Peking,Sumber Vitamin


The table above is easier to understand so that we can directly save/export in the form of an excel file.

In [20]:
df_sol.to_excel("Choosing_Vendor.xlsx")

We can also do a groupby on vendors to see which vendors are the most chosen by the agency. From the groupby results, Sumber Vitamin are the most chosen/reliable for ordering and food service.


In [21]:
df_sol.groupby('Vendor').count()

,Menu
Vendor,
Katineung,1
Sami Asih Group,1
Sangkuriang,3
Sumber Vitamin,4


### Closing
This is a short tutorial to solve GAP problems using pulp. Sorry if there are shortcomings, thank you so much. Good luck :)

